In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
# create a function that can be reused for scraping the data
def get_html_data(url):
    response = requests.get(url)
    return BeautifulSoup(response.text, "html5lib")

### Super Bowl Winners
### Source: Topend Sports
* The Super Bowl is the annual championship game of the National Football League (NFL). 
* Below is a list of all the winners of the Super Bowl since Green Bay won the first in 1967. 
* The years listed indicate the year the Super Bowl was played, the regular season is played during the year before.

In [3]:
# Superbowl winning teams with scores 
sb_url = 'https://www.topendsports.com/events/super-bowl/winners-list.htm'
sb_response = get_html_data(sb_url)

# print(sb_response.prettify())

# results are returned as an iterable list
sb_results = sb_response.find_all('table', class_="list")

# print(sb_results[0].prettify())

# read html to convert the data to dataframe - but this return a list
sb_df = pd.read_html(str(sb_results))

# convert the list to a dataframe
sb_df = sb_df[0]

# display the first 5 rows of dataframe
sb_df.head()

,Year,No.,Winner,Opposition,Score,Venue
0,2020,LIV,Kansas City Chiefs,San Francisco 49ers,31-20,Miami
1,2019,LIII,New England Patriots,Los Angeles Rams,13-3,Atlanta
2,2018,LII,Philadelphia Eagles,New England Patriots,41-33,Minnesota
3,2017,LI,New England Patriots,Atlanta Falcons,34-28,Texas
4,2016,L,Denver Broncos,Carolina Panthers,24-10,California


In [4]:
# rename columns
sb_df = sb_df.rename(columns={"No." : 'sb_no', 
                              'Winner': "team_won", 
                              "Opposition" : "team_lost",
                              'Venue' : 'stadium'}) 
sb_df.head()

,Year,sb_no,team_won,team_lost,Score,stadium
0,2020,LIV,Kansas City Chiefs,San Francisco 49ers,31-20,Miami
1,2019,LIII,New England Patriots,Los Angeles Rams,13-3,Atlanta
2,2018,LII,Philadelphia Eagles,New England Patriots,41-33,Minnesota
3,2017,LI,New England Patriots,Atlanta Falcons,34-28,Texas
4,2016,L,Denver Broncos,Carolina Panthers,24-10,California


In [5]:
# Data has some special characters in Score field. But we need to retain the '-' as is to split the score into 2 fields
sb_df['Score'] = sb_df['Score'].str.replace(r"[^a-zA-Z0-9]+", '-')
sb_df.head()

,Year,sb_no,team_won,team_lost,Score,stadium
0,2020,LIV,Kansas City Chiefs,San Francisco 49ers,31-20,Miami
1,2019,LIII,New England Patriots,Los Angeles Rams,13-3,Atlanta
2,2018,LII,Philadelphia Eagles,New England Patriots,41-33,Minnesota
3,2017,LI,New England Patriots,Atlanta Falcons,34-28,Texas
4,2016,L,Denver Broncos,Carolina Panthers,24-10,California


In [6]:
# split the score in to 2 fields: score_won and score_lost
def get_score_won(Score):
    return Score.split("-")[0]

def get_score_lost(Score):
    return Score.split("-")[1]

# Use the labda function to apply the above function to split the score in the dataframe
sb_df['score_won'] = sb_df['Score'].apply(lambda x: f"{get_score_won(x)}")
sb_df['score_lost'] = sb_df['Score'].apply(lambda x: f"{get_score_lost(x)}")

sb_df.head()

,Year,sb_no,team_won,team_lost,Score,stadium,score_won,score_lost
0,2020,LIV,Kansas City Chiefs,San Francisco 49ers,31-20,Miami,31,20
1,2019,LIII,New England Patriots,Los Angeles Rams,13-3,Atlanta,13,3
2,2018,LII,Philadelphia Eagles,New England Patriots,41-33,Minnesota,41,33
3,2017,LI,New England Patriots,Atlanta Falcons,34-28,Texas,34,28
4,2016,L,Denver Broncos,Carolina Panthers,24-10,California,24,10


In [7]:
# As we split the score into 2 different columns we don't need original score column
sb_df.drop(['Score', 'Year'],axis='columns',inplace=True)

# set index to sb_no 
sb_df.set_index("sb_no", inplace=True)
sb_df.head()

,team_won,team_lost,stadium,score_won,score_lost
sb_no,,,,,
LIV,Kansas City Chiefs,San Francisco 49ers,Miami,31,20
LIII,New England Patriots,Los Angeles Rams,Atlanta,13,3
LII,Philadelphia Eagles,New England Patriots,Minnesota,41,33
LI,New England Patriots,Atlanta Falcons,Texas,34,28
L,Denver Broncos,Carolina Panthers,California,24,10


### Source: ESPN
### NFL History - Super Bowl MVPs

In [8]:
mvp_url = 'http://www.espn.com/nfl/superbowl/history/mvps'
mvp_response = get_html_data(mvp_url)
# print(mvp_response.prettify())

# results are returned as an iterable list
mvp_results = mvp_response.find_all('table', class_="tablehead")

# print(mvp_results[0].prettify())      

# read html and convert it to pandas dataframe
mvp_df = pd.read_html(str(mvp_results))
mvp_df = mvp_df[0]

# rename columns
mvp_df = mvp_df.rename(columns={0: "sb_no", 1 : "mv_player", 2 : "highlights"}) 

# set the value 50 to letter L to be consistent with other super bowl numbers
mvp_df['sb_no'] = mvp_df['sb_no'].replace('50','L', regex=True)
mvp_df.head()

,sb_no,mv_player,highlights
0,Super Bowl Most Valuable Players,Super Bowl Most Valuable Players,Super Bowl Most Valuable Players
1,NO.,PLAYER,HIGHLIGHTS
2,I,"Bart Starr, QB, Green Bay",Two touchdown passes
3,II,"Bart Starr, QB, Green Bay","202 yards passing, 1 TD"
4,III,"Joe Namath, QB, New York Jets",206 yards passing


In [9]:
# set index to sb_no
mvp_df.set_index("sb_no", inplace=True)

# drop the unwanted data
mvp_df.drop(['Super Bowl Most Valuable Players', 'NO.'], inplace=True)

#dispaly the data
mvp_df.head()

,mv_player,highlights
sb_no,,
I,"Bart Starr, QB, Green Bay",Two touchdown passes
II,"Bart Starr, QB, Green Bay","202 yards passing, 1 TD"
III,"Joe Namath, QB, New York Jets",206 yards passing
IV,"Len Dawson, QB, Kansas City","142 yards passing, 1 TD"
V,"Chuck Howley, LB, Dallas","Two interceptions, fumble recovery"


In [10]:
# Split the mvp_player column into name of the player, role, and the team
def get_name(mv_player):
    return mv_player.split(",")[0]

def get_role(mv_player):
    return mv_player.split(",")[1]

def get_team(mv_player):
    return mv_player.split(",")[2]

mvp_df['name'] = mvp_df['mv_player'].apply(lambda x: f"{get_name(x)}")
mvp_df['role'] = mvp_df['mv_player'].apply(lambda x: f"{get_role(x)}")
mvp_df['team'] = mvp_df['mv_player'].apply(lambda x: f"{get_team(x)}")

mvp_df.head()

,mv_player,highlights,name,role,team
sb_no,,,,,
I,"Bart Starr, QB, Green Bay",Two touchdown passes,Bart Starr,QB,Green Bay
II,"Bart Starr, QB, Green Bay","202 yards passing, 1 TD",Bart Starr,QB,Green Bay
III,"Joe Namath, QB, New York Jets",206 yards passing,Joe Namath,QB,New York Jets
IV,"Len Dawson, QB, Kansas City","142 yards passing, 1 TD",Len Dawson,QB,Kansas City
V,"Chuck Howley, LB, Dallas","Two interceptions, fumble recovery",Chuck Howley,LB,Dallas


### Source: Wikipedia
### List of Super Bowl starting quarterbacks

In [11]:
qb_url = 'https://en.wikipedia.org/wiki/List_of_Super_Bowl_starting_quarterbacks'

qb_response = get_html_data(qb_url)
# print(qb_response.prettify())

# results are returned as an iterable list
qb_results = qb_response.find_all('table', class_="wikitable")

# print(qb_results[1].prettify())      

qb_df = pd.read_html(str(qb_results[1]))
qb_df = qb_df[0]
qb_df.head()

,Season,Super Bowl,Winning QB,Team,Losing QB,Team.1
0,1966 AFL/NFL,I,Bart Starr*MVP,Green Bay Packersn,Len Dawson*,Kansas City Chiefsa
1,1967 AFL/NFL,II,Bart Starr*MVP,Green Bay Packersn,Daryle Lamonica,Oakland Raidersa
2,1968 AFL/NFL,III,Joe Namath*MVP,New York Jetsa,Earl Morrall,Baltimore Coltsn
3,1969 AFL/NFL,IV,Len Dawson*MVP,Kansas City Chiefsa,Joe Kapp,Minnesota Vikingsn
4,1970,V,Johnny Unitas*,Baltimore ColtsA,Craig Morton,Dallas CowboysN


In [12]:
# remove the special charachaters from data
qb_df['qb_won'] = qb_df['Winning QB'].str.replace('\W', ' ')
qb_df['qb_lost'] = qb_df['Losing QB'].str.replace('\W', ' ')
qb_df.head()

,Season,Super Bowl,Winning QB,Team,Losing QB,Team.1,qb_won,qb_lost
0,1966 AFL/NFL,I,Bart Starr*MVP,Green Bay Packersn,Len Dawson*,Kansas City Chiefsa,Bart Starr MVP,Len Dawson
1,1967 AFL/NFL,II,Bart Starr*MVP,Green Bay Packersn,Daryle Lamonica,Oakland Raidersa,Bart Starr MVP,Daryle Lamonica
2,1968 AFL/NFL,III,Joe Namath*MVP,New York Jetsa,Earl Morrall,Baltimore Coltsn,Joe Namath MVP,Earl Morrall
3,1969 AFL/NFL,IV,Len Dawson*MVP,Kansas City Chiefsa,Joe Kapp,Minnesota Vikingsn,Len Dawson MVP,Joe Kapp
4,1970,V,Johnny Unitas*,Baltimore ColtsA,Craig Morton,Dallas CowboysN,Johnny Unitas,Craig Morton


In [13]:
qb_df.drop(['Winning QB', 'Losing QB'],axis='columns',inplace=True)
qb_df.head()

,Season,Super Bowl,Team,Team.1,qb_won,qb_lost
0,1966 AFL/NFL,I,Green Bay Packersn,Kansas City Chiefsa,Bart Starr MVP,Len Dawson
1,1967 AFL/NFL,II,Green Bay Packersn,Oakland Raidersa,Bart Starr MVP,Daryle Lamonica
2,1968 AFL/NFL,III,New York Jetsa,Baltimore Coltsn,Joe Namath MVP,Earl Morrall
3,1969 AFL/NFL,IV,Kansas City Chiefsa,Minnesota Vikingsn,Len Dawson MVP,Joe Kapp
4,1970,V,Baltimore ColtsA,Dallas CowboysN,Johnny Unitas,Craig Morton


In [14]:
# rename columns
qb_df = qb_df.rename(columns={"Season" : "year", "Super Bowl" : 'sb_no', 'Team': "team_won", "Team.1" : "team_lost"}) 
qb_df.head()

,year,sb_no,team_won,team_lost,qb_won,qb_lost
0,1966 AFL/NFL,I,Green Bay Packersn,Kansas City Chiefsa,Bart Starr MVP,Len Dawson
1,1967 AFL/NFL,II,Green Bay Packersn,Oakland Raidersa,Bart Starr MVP,Daryle Lamonica
2,1968 AFL/NFL,III,New York Jetsa,Baltimore Coltsn,Joe Namath MVP,Earl Morrall
3,1969 AFL/NFL,IV,Kansas City Chiefsa,Minnesota Vikingsn,Len Dawson MVP,Joe Kapp
4,1970,V,Baltimore ColtsA,Dallas CowboysN,Johnny Unitas,Craig Morton


In [15]:
qb_df = qb_df.replace('MVP','', regex=True)
qb_df = qb_df.replace('AFL/NFL','', regex=True)

qb_df.head()

,year,sb_no,team_won,team_lost,qb_won,qb_lost
0,1966,I,Green Bay Packersn,Kansas City Chiefsa,Bart Starr,Len Dawson
1,1967,II,Green Bay Packersn,Oakland Raidersa,Bart Starr,Daryle Lamonica
2,1968,III,New York Jetsa,Baltimore Coltsn,Joe Namath,Earl Morrall
3,1969,IV,Kansas City Chiefsa,Minnesota Vikingsn,Len Dawson,Joe Kapp
4,1970,V,Baltimore ColtsA,Dallas CowboysN,Johnny Unitas,Craig Morton


In [16]:
# the last character of the team name has 'a' or 'n' to nitity afc or nfc. Remove that extra character from team names 
qb_df['team_won'] = [sub[ : -1] for sub in qb_df['team_won']] 
qb_df['team_lost'] = [sub[ : -1] for sub in qb_df['team_lost']]

# Update the super bowl number 50 to L
qb_df['sb_no'] = qb_df['sb_no'].replace('50','L', regex=True)

qb_df.head()

,year,sb_no,team_won,team_lost,qb_won,qb_lost
0,1966,I,Green Bay Packers,Kansas City Chiefs,Bart Starr,Len Dawson
1,1967,II,Green Bay Packers,Oakland Raiders,Bart Starr,Daryle Lamonica
2,1968,III,New York Jets,Baltimore Colts,Joe Namath,Earl Morrall
3,1969,IV,Kansas City Chiefs,Minnesota Vikings,Len Dawson,Joe Kapp
4,1970,V,Baltimore Colts,Dallas Cowboys,Johnny Unitas,Craig Morton


In [17]:
# set index to sb_no
qb_df.set_index("sb_no", inplace=True)
qb_df.tail()

,year,team_won,team_lost,qb_won,qb_lost
sb_no,,,,,
LI,2016,New England Patriots,Atlanta Falcons,Tom Brady,Matt Ryan
LII,2017,Philadelphia Eagles,New England Patriots,Nick Foles,Tom Brady
LIII,2018,New England Patriots,Los Angeles Rams,Tom Brady,Jared Goff
LIV,2019,Kansas City Chiefs,San Francisco 49ers,Patrick Mahomes,Jimmy Garoppolo
Super Bowl,Season,Tea,Tea,Winning QB,Losing QB


In [18]:
qb_df.drop(['Super Bowl'], inplace=True)
#dispaly the data
qb_df.tail()

,year,team_won,team_lost,qb_won,qb_lost
sb_no,,,,,
L,2015,Denver Broncos,Carolina Panthers,Peyton Manning,Cam Newton
LI,2016,New England Patriots,Atlanta Falcons,Tom Brady,Matt Ryan
LII,2017,Philadelphia Eagles,New England Patriots,Nick Foles,Tom Brady
LIII,2018,New England Patriots,Los Angeles Rams,Tom Brady,Jared Goff
LIV,2019,Kansas City Chiefs,San Francisco 49ers,Patrick Mahomes,Jimmy Garoppolo


In [19]:
print('Number of rows in the dataset: ', qb_df.shape[0])
print('Number of columns in the dataset: ', qb_df.shape[1])

Number of rows in the dataset:  54
Number of columns in the dataset:  5
